## Домашняя работа
ФИО: Гуламов Артём Садриддинович

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`


### проверка задания 1

In [7]:

from threading import Lock

class ThreadSafeSingleton(type):
    _instances = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
        # Двойная проверка для оптимизации (Double-Checked Locking)
        if cls not in cls._instances:
            with cls._lock:
                if cls not in cls._instances:
                    cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


In [8]:
class DatabasePool(metaclass=ThreadSafeSingleton):
    def __init__(self):
        self.connections = []
        # Имитация создания соединений
        for i in range(5):
            self.connections.append(f"Connection_{i}")
        self.next_connection_index = 0
    
    def get_connection(self):
        if not self.connections:
            return None
        
        conn = self.connections[self.next_connection_index % len(self.connections)]
        self.next_connection_index += 1
        return conn

In [9]:
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()


assert pool1 is pool2 is pool3
print("Все экземпляры ссылаются на один объект")


conn1 = pool1.get_connection()
conn2 = pool2.get_connection()
assert conn1 != conn2
print("Соединения корректно разделяются между экземплярами")

Все экземпляры ссылаются на один объект
Соединения корректно разделяются между экземплярами


## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

### проверка задания 2

In [10]:
class SimpleCountMeta(type):
    _counters = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._counters:
            cls._counters[cls] = 0
        cls._counters[cls] += 1
        return super().__call__(*args, **kwargs)
    
    def get_count(cls):
        return cls._counters.get(cls, 0)



In [11]:
class User(metaclass=SimpleCountMeta):
    def __init__(self, name):
        self.name = name

class Product(metaclass=SimpleCountMeta):
    def __init__(self, name):
        self.name = name


In [12]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

### проверка задания 3

In [ ]:
class DescribeMeta(type):
    
    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        
        def describe(self):
            return f"Это объект класса {name}"
        
        cls.describe = describe

In [14]:
class Car(metaclass=DescribeMeta):
    def __init__(self, brand):
        self.brand = brand
    
    def __str__(self):
        return f"Car({self.brand})"


class Book(metaclass=DescribeMeta):
    def __init__(self, title):
        self.title = title
    
    def __str__(self):
        return f"Book({self.title})"

In [15]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

### проверка задания 4

In [16]:
class SaveMeta(type):
    
    def __new__(cls, name, bases, attrs):
        # Проверяем, есть ли метод save в атрибутах класса
        if 'save' not in attrs:
            raise TypeError(f"Класс {name} должен иметь метод save()")
        
        # Проверяем, что save - это callable метод (функция)
        save_method = attrs['save']
        if not callable(save_method):
            raise TypeError(f"Метод save() в классе {name} должен быть вызываемым")
        
        return super().__new__(cls, name, bases, attrs)

In [17]:
class User(metaclass=SaveMeta):
    def __init__(self, name):
        self.name = name
    
    def save(self):
        print(f"Пользователь {self.name} сохранен в базе данных")

In [ ]:
print("Создаем User с методом save():")
user = User("Alice")
user.save()  # Должно работать

Создаем User с методом save():
Пользователь Alice сохранен в базе данных


In [ ]:
print("\nПопытка создать класс Message без метода save():")
try:
    class Message(metaclass=SaveMeta):
        def __init__(self, text):
            self.text = text
    
    message = Message("Hello")  # Эта строка не должна выполниться

except TypeError as e:
    print(f"Ошибка: {e}")


Попытка создать класс Message без метода save():
Ошибка: Класс Message должен иметь метод save()
